<a href="https://www.kaggle.com/code/gabripo93/the-perfect-match-for-your-tech-and-business-needs?scriptVersionId=210004674" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Role-based reasoning to Find the Right Company and Generate Clause-by-Clause Reports for Tenders

-> A *tender* is a formal process, where organizations or companies invite suppliers, contractors, or service providers to submit proposals or bids to deliver a specified project, product, or service.
-> A *clause-by-clause* is a detailed examination of this tender, that highlights the compliant and not-compliant requirements with respect to the company offer.

As this process is carried out in most of the companies and it is quite time-consuming, we believe that the Gemini's long context window introduces a novelty in this business process:

- It analyze long technical and commercial tenders for a project.
- It assesses the compatibility of companies' products and services with tender documents.
- It *finds the best company and product-service combination to execute your project*, generating a clause-by-clause report with compliant and non-compliant specifications.

## Notebook Structure 📓

The notebook is divided into different sections, each with a specific objective:

- Dataset load (see *Relevant Project and Future steps* chapter for data generation)
- Tenders for a project are parsed, converting their information into text.
- Information scraped from various companies' websites is loaded as text.
- All text is processed by Gemini using different prompts, combining role-based reasoning, chain of thoughts, Gemini API context caching feature and in-chat memory.

## Role-based Reasoning 🧠

Role-based reasoning is implemented by segmenting tasks and delegating responsibilities to distinct roles. For example:

- **Technical and Commercial Tender Agents**: Separate prompts (tender_prompt_template_technical and tender_prompt_template_commercial) guide the roles of the technical tender engineer and commercial tender manager. Each user has distinct objectives: identifying and summarizing technical or commercial requirements within tenders. 

- A distinct prompt (**Sales Manager"") is also prepared for analyzing companies (e.g., SIEMENS and HITACHI) to match tender requirements with their products and solutions (get_response_companies_info). This allows tailored reasoning for comparing affinities between tenders and company offerings.

## Chain of Thoughts 🧩

In both technical and commercial prompts, we used phrases like "Think step by step" to guide the agent toward incremental reasoning. This ensures that requirements are dissected and analyzed in detail.
The user prompt specifies a structured approach to calculating an affinity score, prompting the agent to explicitly explain the calculation process. Finally, in the Clause-by-Clause Analysis, the final prompt directs the agent to meticulously compare tender requirements with company specifications, maintaining a clear progression in thought. 

## In-chat Memory and caching 🗃️

In-chat memory stores all the interactions from the technical and commercial tender analysis, and from the companies website. It keeps track of the responses from the different roles (e.g., technical engineer, commercial manager, sales manager). This memory allows to build upon the context of earlier prompts without having to constantly reprocess the same information. 
An alternative method with context caching is also tested.
With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.
Stay tuned for the interesting comparison between the two methods.

## Conclusion for the Use Case 

Using a long context window instead of Retrieval-Augmented Generation (RAG) for this use case was particularly beneficial due to the task's nature, which involves reasoning across interdependent documents. The unified context allows the model to cross-reference tender requirements and company offerings directly, ensuring cohesive and accurate analysis. This is particularly advantageous for tasks like affinity scoring, which require simultaneous consideration of multiple data points.
The notebook's approach scales better for handling multiple queries simultaneously, as it avoids the bottleneck of sequential agent calls. For new tender projects, it's only necessary to update the in-chat memory.

*In summary, why did we decide to build this notebook?*

1. **Holistic Context Retention**: By storing the entire history of tender analyses (both technical and commercial) and company solution evaluations, the model retains a comprehensive understanding of all previously provided information. This holistic context allows the model to reason about how specific requirements and offerings interrelate across multiple prompts. In RAG, the system retrieves only the most relevant chunks of information for each query; this efficient approach can lead to fragmented analyses, potentially overlooking interconnections.

2. **Dynamic Role-based Collaboration**: By maintaining a long context, the system allows outputs from technical engineers, commercial managers, and sales managers to flow into a unified reasoning framework. A long context window naturally informs each role, creating a seamless chain of thought.

3. **Reduced Query Overhead**: Long context windows reduce the need for multiple retrieval calls, making the process more efficient in scenarios where information is revisited or refined iteratively. RAG introduces latency and computational costs because each query requires searching and ranking document chunks. 

4. **Affinity Score Calculation**: Computing an affinity score across companies for tenders requires integrating technical and commercial analysis alongside company data. This step benefits significantly from the model's ability to access all previous responses simultaneously.

5. **Clause-by-Clause Compliance Analysis**: Clause-by-clause analysis relies on cross-referencing previously extracted requirements with company offerings. The long context window with conteoxt caching allows the model to directly reference earlier inputs and outputs without reloading or retrieving.


## Conclusion 

#### The long context window acts as a shared workspace, recording and making all roles outputs accessible for seamless and holistic reasoning. In today's interconnected world, where partnerships and synergies are essential to addressing complex challenges, we envision a tool that enables continuous reasoning, uncovers new patterns and solutions, and minimizes the fragmentation of insights.




### Related Projects and Future steps 🔍💡

- The data generation and cleaning is performed with another repo stored in github:  https://github.com/gabripo/kaggle-gemini-long-context.

- In the past few months, we also implemented a multi-agent framework (LumadaAI) using LangChain and OpenAI, where each company was represented by a dedicated agent.
  **LumadaAI** is publicly available at https://github.com/SecchiAlessandro/LumadaAI.
  This framework featured a supervisor agent that dynamically routed user queries to the most relevant company-specific agent based on the query context. While innovative, this approach faced challenges in stability, accuracy, and efficiency, making the current solution more effective. As agents operated independently, generating combined solutions from different companies was difficult. Additionally, for each query, the supervisor needed to perform additional reasoning before invoking an agent. If a query was relevant to multiple agents, the framework had to perform sequential calls, compounding latency. The current solution with centralized reasoning ensures consistent application of logic and context.

- **EasyRAG** (https://github.com/gabripo/easyrag) is another RAG tool that performs RAG over locally stored documents. We are benchmarking this tool with Gemini's long context window: adding one or more PDFs to Gemini's context window could provide more precise insights than the RAG approach.

- In the next weeks we are curious to exploit the multimodal potential of Gemini. With a similar concept presented in this notebook, we want to use an input video of a job candidate's attitude and skills to identify the best matches or highlight differences compared to a video from the company's professional and behavioral requirements.


In [1]:
# import Python libraries
import os
import json
from IPython.display import Markdown

In [2]:
# auxiliary function to read JSON files
def read_json_info(jsonFilePath: str) -> dict:
    if os.path.exists(jsonFilePath):
        with open(jsonFilePath, "r") as f:
            data = json.load(f)
        return data
    else:
        return {}

In [3]:
# auxiliary Python decorator to execute a function again, if its execution fails
# this is helpful when calling the Gemini's API since Gemini has a rate limiter and, if an execution fails for that, there will be some waiting time before retrying
import time

def retry_on_failure(wait_time_seconds=60, max_retries=5):
    def decorator_retry(func):
        
        def wrapper_retry(*args, **kwargs):
            retries = 0
            while retries < max_retries:
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    retries += 1
                    if retries < max_retries:
                        print(
                            f"Function failed with error: {e}. Retrying in {wait_time_seconds} seconds... (Attempt {retries}/{max_retries})"
                        )
                        time.sleep(wait_time_seconds)
                    else:
                        print(f"Function failed after {max_retries} attempts.")
                        raise e
        return wrapper_retry

    return decorator_retry

In [4]:
dataset_path = '/kaggle/input/tenders-and-companies-websites'
working_path = '/kaggle/working'

In [5]:
!mkdir -p /kaggle/working/tenders
tenders_working_path = os.path.join(working_path, 'tenders')

!mkdir -p /kaggle/working/companies
companies_working_path = os.path.join(working_path, 'companies')

# Build a chat with Gemini

In [6]:
# API key got here: https://ai.google.dev/tutorials/setup

import google.generativeai as genai
from kaggle_secrets import UserSecretsClient


user_secrets = UserSecretsClient()
secret_key = user_secrets.get_secret("GEMINI_API_KEY")

genai.configure(api_key = secret_key)

model_name = 'gemini-1.5-flash'
model = genai.GenerativeModel(model_name=model_name)

model_info = genai.get_model(f"models/{model_name}")
print(f"{model_info.input_token_limit=}")
print(f"{model_info.output_token_limit=}")

model_info.input_token_limit=1000000
model_info.output_token_limit=8192


In [7]:
print("List of models that support generateContent:\n")
for m in genai.list_models():
    if "generateContent" in m.supported_generation_methods:
        print(m.name)

List of models that support generateContent:

models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-pro-exp-0827
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-exp-0827
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/learnlm-1.5-pro-experimental
models/gemini-exp-1114
models/gemini-exp-1121


In [8]:
# the decorator ensures that, if an error occurs, the function will be executed again
@retry_on_failure(wait_time_seconds=60, max_retries=5)
def ask_gemini(prompt, model=None, history=[], model_name = 'gemini-1.5-flash'):
    """
    function to call Gemini, providing chat history
    if a chat is already available, it will be used
    """
    if model == None:
        model = genai.GenerativeModel(model_name=model_name)
        
    chat = model.start_chat(history=history)
    
    response = chat.send_message(prompt)
    return response

In [9]:
# initialize the response dictionary
responses = {}

# Analyze the tenders

In [10]:
# read the json file related to tenders from the input dataset
tenders_info_json_path = os.path.join(dataset_path, 'tenders_info.json')
tenders_info = read_json_info(tenders_info_json_path)

# tenders_info is a dictionary, where the key is the name of the tender file and the related value its information
# print(tenders_info["tender_wind.pdf"])

In [11]:
# list the processed tender files
tenders = tenders_info.keys()

In [12]:
tender_prompt_template_technical = """
You are an experienced technical tender engineer. 
The document you have is a tender, that contains also technical requirements for a project.
Think step by step on how to look for the relevant technical requirements and make a detailed summary.
The content of the document is: """
tender_prompts_technical = []
for info in tenders_info.values():
    tender_prompts_technical.append(f"You have a document called {info['name']} . " + tender_prompt_template_technical + f"{info['content']}")

In [13]:
tender_prompt_template_commercial = """
You are an experienced commercial tender manager. 
The document you have is a tender, that contains also commercial requirements for a project.
Think step by step on how to look for the relevant commercial requirements and make a detailed summary.
The content of the document is: "
"""
tender_prompts_commercial = []
for info in tenders_info.values():
    tender_prompts_commercial.append(f"You have a document called {info['name']} . " + tender_prompt_template_commercial + f"{info['content']}")

In [14]:
@retry_on_failure(wait_time_seconds=60)
def get_responses_tenders(subject, tender_prompts):
    tenders_json_file_path = os.path.join(tenders_working_path, f'tenders_{subject}.json')
    
    if os.path.exists(tenders_json_file_path):
        responses = read_json_info(tenders_json_file_path)
        print(f"tender_{subject}: Responses loaded from file {tenders_json_file_path}")
    else:
        responses = {}
        for tender_prompt, tender_name in zip(tender_prompts, tenders):
            print(f"tender_{subject}: Generating response for tender {tender_name} ...")
            response = ask_gemini(prompt = tender_prompt)
            #print(response.text)

            responses[tender_name] = {'prompt': tender_prompt, 'answer': response.text}
            print(f"tender_{subject}: Response for tender {tender_name} generated.")
    
        with open(tenders_json_file_path, 'w') as f:
            json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"tender_{subject}: Responses stored into {tenders_json_file_path}")
    
    print(f"tender_{subject}: Analysis concluded!\n")
    return responses

# each call of get_responses_tenders() will generate a tenders_{subject}.json file
# each generated file so will contain the Gemnini's responses for a given subject
responses['tender_technical'] = get_responses_tenders("technical", tender_prompts_technical)
responses['tender_commercial'] = get_responses_tenders("commercial", tender_prompts_commercial)

tender_technical: Generating response for tender tender_wind.pdf ...
tender_technical: Response for tender tender_wind.pdf generated.
tender_technical: Generating response for tender tender_solar.pdf ...
tender_technical: Response for tender tender_solar.pdf generated.
tender_technical: Responses stored into /kaggle/working/tenders/tenders_technical.json
tender_technical: Analysis concluded!

tender_commercial: Generating response for tender tender_wind.pdf ...
tender_commercial: Response for tender tender_wind.pdf generated.
tender_commercial: Generating response for tender tender_solar.pdf ...
tender_commercial: Response for tender tender_solar.pdf generated.
tender_commercial: Responses stored into /kaggle/working/tenders/tenders_commercial.json
tender_commercial: Analysis concluded!



# Analyze the companies products and solutions

## Data generation
Information about interesting companies is obtained from their websites.

To generate data out of the companies' websites, we implemented a crawler.

The final output of the crawler is a JSON file, in which each field refers to a company: for each company, all the information of the websites is merged.

> The generation of information can be found in the Kaggle Notebook https://www.kaggle.com/code/gabripo93/gemini15-long-context-competition-generate-dataset

### Details about the crawling process:
- **Recursive scan**: after a webpage is scanned and its content is stored, eventual found sublinks are scanned, as well. A limit of the wepages to download is given as input.
- **Redundant information is deleted**: if some website content can be found multiple times in all the webpages of one company, then it is skipped. *Example*: undesired and redundant lines like "Contact Us" are removed, ensuring that the final content does not include unnecessary sentences.
- **Caching of already downloaded pages**: for each webpage, the content is stored in a JSON file, as well as the found sublinks. *Example*: after a run with a limit of N pages, other runs with less than N pages will use the stored files instead downloading data from internet; at the contrary, if the limit is increased to M > N pages, only M - N additional pages will be downloaded while the first N pages will be taken from the stored file. 

In [15]:
companies_info_json_path = os.path.join(dataset_path, 'companies_info.json')
companies_info = read_json_info(companies_info_json_path)

# companies_info is a dictionary, where the key is the name of the company and the related value its information
# print(companies_info["SIEMENS"]) 

In [16]:
@retry_on_failure(wait_time_seconds=60)
def get_response_companies(company_name):
    companies_json_file_path = os.path.join(companies_working_path, f'companies_{company_name}.json')
    
    if os.path.exists(companies_json_file_path):
        responses = read_json_info(companies_json_file_path)
        print(f"companies_{company_name}: Responses loaded from file {companies_json_file_path}")
    else:
        print(f"companies_{company_name}: Generating response for company {company_name} ...")
        responses = {}
        company_prompt = f"These are the information of products and solutions for the company {company_name} : {companies_info[company_name]}"
        response = ask_gemini(prompt = company_prompt)
        responses[company_name] = {'prompt': company_prompt, 'answer': response.text}

        with open(companies_json_file_path, 'w') as f:
                json.dump(responses, f, ensure_ascii=True, indent=4)
        print(f"companies_{company_name}: Responses stored into {companies_json_file_path}")

    print(f"companies_{company_name}: Response for company {company_name} generated!")
    return responses

# each call of get_responses_companies() will generate a companies_{company_name}.json file
# each generated file so will contain the Gemnini's responses for a given company
# the purpose of generating responses given companies information is to store it in the chat history


In [17]:
responses['company_hitachi'] = get_response_companies("HITACHI")

companies_HITACHI: Generating response for company HITACHI ...
companies_HITACHI: Responses stored into /kaggle/working/companies/companies_HITACHI.json
companies_HITACHI: Response for company HITACHI generated!


In [18]:
responses['company_siemens'] = get_response_companies("SIEMENS")

companies_SIEMENS: Generating response for company SIEMENS ...
companies_SIEMENS: Responses stored into /kaggle/working/companies/companies_SIEMENS.json
companies_SIEMENS: Response for company SIEMENS generated!


# Build a chat history based on previous prompts

In [19]:
# example how to include the chat history here https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/getting-started/intro_gemini_chat.ipynb
# description of the Content class here https://github.com/google-gemini/generative-ai-python/blob/main/docs/api/google/generativeai/GenerativeModel.md
from google.generativeai.protos import Content, Part

history_chat = []

def add_history_to_chat_single(response, user, history_chat):
    query = Part()
    query.text = f"{user}: {response['prompt']}"
    history_chat.append(Content(role="user", parts=[query]))

    answer = Part()
    answer.text = response['answer']
    history_chat.append(Content(role="model", parts=[answer]))
    return

def add_history_to_chat(responses, user, history_chat):
    for response in responses.values():
        add_history_to_chat_single(response, user, history_chat)
    return 

add_history_to_chat(responses['tender_technical'], "technical engineer", history_chat)
add_history_to_chat(responses['tender_commercial'], "commercial manager", history_chat)
add_history_to_chat(responses['company_siemens'], "sales manager for siemens", history_chat)
add_history_to_chat(responses['company_hitachi'], "sales manager for hitachi", history_chat)

## Test the chat history

In [21]:
prompt_roles = "Which are the roles given in the prompt from the user? There are only two for tenders and one for company"
responses['prompt_roles'] = ask_gemini(prompt=prompt_roles, history=history_chat)

Markdown(responses['prompt_roles'].text)

The prompt from the user assigns three distinct roles:

1. **Technical Engineer (for tenders):** This role focuses on evaluating the technical specifications and requirements within a tender document.

2. **Commercial Manager (for tenders):** This role centers on analyzing the commercial aspects and requirements within a tender document, such as pricing, payment terms, and risk mitigation strategies.

3. **Sales Manager (for a specific company, Siemens):** This role requires summarizing product information and solutions offered by a particular company (Siemens) based on provided website content.


In [22]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_roles, 'answer': responses['prompt_roles'].text}, "technical engineer", history_chat)

# Find the most suitable company

In [23]:
prompt_match = """
1. For company SIEMENS and HITACHI, find the respective relevant products and solutions with respect to the analyzed tenders.
   The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   When you refer to a source in the text you have, report the related URL if specified - it should be embedded in the text.
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1. Explain the way how you computed this percentage.
"""

Markdown(prompt_match)


1. For company SIEMENS and HITACHI, find the respective relevant products and solutions with respect to the analyzed tenders.
   The information is in the form of text I provided, then you do not need to read additional documents or access to websites.
   When you refer to a source in the text you have, report the related URL if specified - it should be embedded in the text.
   
2. Calculate an affinity score in percentage for each company based on analysis in point 1. Explain the way how you computed this percentage.


In [24]:
print("Finding the most suitable company for the tenders ...")
responses['prompt_match'] = ask_gemini(prompt=prompt_match, history=history_chat)
print("Response to the prompts is ready!")

Markdown(responses['prompt_match'].text)

Finding the most suitable company for the tenders ...
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 1/5)
Function failed with error: 429 Resource has been exhausted (e.g. check quota).. Retrying in 60 seconds... (Attempt 2/5)
Response to the prompts is ready!


To analyze Siemens and Hitachi's offerings in relation to the solar and wind tenders, we need to revisit the tender requirements.  Since the prompt doesn't directly provide the specific requirements, I'll assume a generalized set of needs common to both types of renewable energy projects based on the previous responses.  This includes items like power generation, energy storage, grid connection, automation/control, and digital services for monitoring and maintenance.

**I.  Relevant Products and Solutions:**

**A. Siemens:**

* **Wind Energy:**
    * **Gas Turbines:**  The SGT-50 (up to 2 MW), SGT-400 (10-15 MW), SGT5-8000H (up to 450 MW), SGT5-9000HL (up to 593 MW), and SGT6-5000F (up to 260 MW) are relevant depending on project scale.  These offer fuel flexibility (including hydrogen), fast start-up, and low emissions. (URLs for each model were provided in the original prompt).
    * **Generators:**  SGen series generators, adaptable to different turbine types and power ratings.
    * **High-Voltage Substations:** For grid connection, including AIS and GIS options and prefabricated solutions. (URL provided).
    * **Digital Solutions (Omnivise Portfolio):** Omnivise Asset Management, Omnivise Performance Solutions, Omnivise Hybrid Control, and Omnivise Electrical Solutions, and the GT Auto Tuner for optimization and remote monitoring. (URLs provided).
    * **Lifecycle Services:**  For long-term support and maintenance of turbines, generators, and substations. (URL provided).
* **Solar Energy:**
    *  Siemens' offerings in solar energy are less directly apparent in the provided text.

**B. Hitachi Energy:**

* **Wind Energy:**
    * **High-Voltage Switchgear:**  Includes AIS and GIS options, potentially including SF6-free "Blue" technology. (URLs provided).
    * **Transformers:**  For step-up and grid connection. (URL provided).
    * **Reactors:** Shunt reactors for reactive power compensation, potentially including variable shunt reactors. (URL provided).
    * **Surge Arresters:** For overvoltage protection. (URL provided).
    * **Digital Substation Solutions:** Increasing controllability, renewable integration, and safety.  (URL provided).
    * **Grid-eXpand™:** Modular grid connection solutions for faster deployment and cost savings. (URL provided).
* **Solar Energy:**
    * **High-Voltage Switchgear:**  Similar to wind energy applications,  AIS, GIS, and “Blue” solutions are potentially relevant.  (URLs provided).
    * **Transformers:**  For step-up and grid connection. (URL provided).
    * **Digital Substation Solutions:**  Also relevant for solar energy grid integration. (URL provided).
    * **Grid-eXpand™:** Again, a relevant solution for efficient grid connections. (URL provided)


**II. Affinity Score Calculation:**

To calculate the affinity score, we'll assign points to each company based on the number of relevant product categories mentioned.  I'll use a simplified system for demonstration; a more robust scoring system would involve weighting different product categories based on their importance in each tender.  Let's assume the following weights for categories:

* **Power Generation (Wind/Solar):** 3 points (essential)
* **Energy Storage:** 2 points (highly important)
* **Grid Connection:** 2 points (highly important)
* **Automation/Control:** 2 points (highly important)
* **Digital Services:** 2 points (highly important)


**A. Siemens Scoring:**

* **Power Generation (Wind):** 3 points (Gas Turbines)
* **Grid Connection:** 2 points (High-Voltage Substations)
* **Automation/Control:** 2 points (Omnivise Portfolio)
* **Digital Services:** 2 points (Omnivise Portfolio, Lifecycle Services)
* **Power Generation (Solar):** 0 points (not explicitly mentioned)
* **Energy Storage:** 0 points (not explicitly mentioned)

**Total Siemens Score:** 9 points


**B. Hitachi Energy Scoring:**

* **Power Generation (Wind):** 0 points (Not explicitly mentioned)
* **Energy Storage:** 0 points (Not explicitly mentioned)
* **Grid Connection:** 2 points (Grid-eXpand™)
* **Automation/Control:** 2 points (Substation Automation, protection and control solutions)
* **Digital Services:** 2 points (Various digital monitoring and management solutions within the text)
* **Power Generation (Solar):** 0 points (Not explicitly mentioned)

**Total Hitachi Energy Score:** 6 points

**Affinity Score Calculation:**

To convert the scores to percentages, we'll assume a maximum possible score for a complete match of all categories is 11 points (5 categories * maximum 2 points + 1 category with 3 points). Therefore:

* **Siemens Affinity Score:** (9/11) * 100% ≈ 82%
* **Hitachi Energy Affinity Score:** (6/11) * 100% ≈ 55%

**III. Explanation:**

The affinity score represents the percentage match between the company's product portfolio and the generalized needs of the tenders.  Siemens scores higher because its portfolio more directly addresses the assumed requirements, particularly in the wind energy sector through offering of turbines, generators, substations and digital solutions.   Hitachi Energy's score is lower, primarily due to its solutions being less directly related to the power generation aspect (though they have strong offerings in grid connection and automation).  This calculation is a simplification; a more sophisticated analysis would require a detailed breakdown of the tender requirements and weighting of different criteria based on their relative importance.


In [25]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_match, 'answer': responses['prompt_match'].text}, "technical engineer", history_chat)

# Generate the clause-by-clause

In [31]:
user_prompt = """

Consider the company with the highest affinity score and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements of the tender with respect to the selected company. For each company, insert the URL of the source where you found information about mentioned products and solutions into the table.
When possible, show data and explain the reasons behind your thinking in tables.
"""

Markdown(user_prompt)



Consider the company with the highest affinity score and return the clause by clause analysis considering technical and commercial compliant and not-compliant requirements of the tender with respect to the selected company. For each company, insert the URL of the source where you found information about mentioned products and solutions into the table.
When possible, show data and explain the reasons behind your thinking in tables.


In [32]:
system_prompt = """
You are an experienced team of business development managers and tender engineers, commercial managers.
You need to create a detailed clause by clause from the tender documentations and the most affine company specifications.
"""

In [33]:
print("Generating the clause by clause ...")
prompt_clause_by_clause = f"{system_prompt} {user_prompt}"
responses['prompt_clause_by_close'] = ask_gemini(prompt=prompt_clause_by_clause, history=history_chat)
print("Response to the prompts is ready!")

Markdown(responses['prompt_clause_by_close'].text)

Generating the clause by clause ...
Response to the prompts is ready!


Based on the previous analysis, Siemens has the higher affinity score (approximately 82%) for the combined wind and solar tender compared to Hitachi Energy (approximately 55%). Therefore, we will focus on a clause-by-clause analysis comparing the (generalized) tender requirements with Siemens' capabilities.  We'll assume a generalized tender encompassing aspects of both wind and solar projects, as specific requirements were not provided.


**I.  Generalized Tender Requirements (Assumed):**

To perform a clause-by-clause analysis, we need to define the assumed tender clauses.  These will be based on common requirements for renewable energy projects, drawing from previous analyses:

| Clause No. | Clause Description                                                                |
|------------|------------------------------------------------------------------------------------|
| 1          | Power Generation System (Capacity, Technology, Efficiency)                         |
| 2          | Energy Storage System (Capacity, Technology, Response Time, Cycle Life)            |
| 3          | Grid Connection (Substation, Transmission Lines, Compliance)                       |
| 4          | Balance of Plant (Electrical Systems, Control Room, Safety Systems)                 |
| 5          | Monitoring and Control System (SCADA, Data Acquisition, Remote Access)            |
| 6          | Warranty and After-Sales Service (Duration, Scope, Spare Parts)                     |
| 7          | Project Management and Implementation (Timeline, Risk Mitigation)                   |
| 8          | Cybersecurity (Compliance, Risk Mitigation)                                        |
| 9          | Environmental Compliance (Emissions, Permitting)                                  |
| 10         | Commercial Requirements (Pricing, Payment Terms, Financing)                        |


**II. Clause-by-Clause Analysis (Siemens):**

This analysis uses a simplified approach; a real tender would necessitate a far more detailed specification.  We will assume Siemens is bidding for both wind and solar components (although the original Siemens data provided was heavier in wind).

| Clause No. | Clause Description                                                                | Siemens Compliance | Siemens Solution(s)                                                       | URL(s)                                                                                    | Justification/Notes                                                                                                        |
|------------|------------------------------------------------------------------------------------|----------------------|---------------------------------------------------------------------------|---------------------------------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------|
| 1          | Power Generation System (Capacity, Technology, Efficiency)                         | Partially Compliant  | SGT-50, SGT-400, SGT5-8000H, SGT5-9000HL, SGT6-5000F (Wind);  Limited solar details  | Multiple URLs from original prompt (various gas turbine models).                                | Siemens offers a wide range of gas turbines suitable for different project scales; solar solutions less defined in provided text. |
| 2          | Energy Storage System (Capacity, Technology, Response Time, Cycle Life)            | Not Compliant        | Not explicitly mentioned in provided text.                                     | None                                                                                       | Siemens' offerings in energy storage were not detailed in the provided text.                                                |
| 3          | Grid Connection (Substation, Transmission Lines, Compliance)                       | Compliant            | High-Voltage Substations, (AIS, GIS)                                       | https://www.siemens-energy.com/global/en/home/products-services/product-offerings/substations.html | Siemens offers comprehensive substation solutions for grid connection.                                                       |
| 4          | Balance of Plant (Electrical Systems, Control Room, Safety Systems)                 | Partially Compliant  | Omnivise Electrical Solutions would cover much of this.                         | Multiple URLs from original prompt (Omnivise Portfolio).                                    |  Detailed balance-of-plant specifics are not explicitly provided but implied in Omnivise's scope.                               |
| 5          | Monitoring and Control System (SCADA, Data Acquisition, Remote Access)            | Compliant            | Omnivise Asset Management, Omnivise T3000                                      | Multiple URLs from original prompt (Omnivise Portfolio and Omnivise T3000).                    | Siemens' Omnivise portfolio offers comprehensive monitoring and control systems with remote access.                           |
| 6          | Warranty and After-Sales Service (Duration, Scope, Spare Parts)                     | Compliant            | Lifecycle Services mentioned, specific durations would need further clarification. | https://www.siemens-energy.com/us/en/home/products-services/service/lifecycle-services.html | Siemens offers lifecycle services, but specific warranty durations and spare parts availability would need to be specified in the bid. |
| 7          | Project Management and Implementation (Timeline, Risk Mitigation)                   | Partially Compliant  |  Implied through service offerings, but specific details are missing.        | None                                                                                       | Siemens' experience is mentioned, but a detailed project plan with risk mitigation strategies would be required in a complete bid. |
| 8          | Cybersecurity (Compliance, Risk Mitigation)                                        | Partially Compliant  |  Mentioned within Omnivise portfolio, but specific compliance details needed. | Multiple URLs from original prompt (Omnivise Portfolio).                                    | Siemens addresses cybersecurity, but specific compliance certifications and mitigation strategies would need detailed specification. |
| 9          | Environmental Compliance (Emissions, Permitting)                                  | Partially Compliant  | Low emission gas turbines, but permitting is not directly addressed.          | Multiple URLs from original prompt (various gas turbine models).                               |  Gas turbines are described as low emission; compliance with specific permitting requirements would need to be outlined in the bid. |
| 10         | Commercial Requirements (Pricing, Payment Terms, Financing)                        | Not Applicable       |  Would be addressed in a separate commercial proposal.                        | None                                                                                       |  Commercial terms are not part of the technical specifications.                                                            |


**III. Conclusion:**

Siemens demonstrates significant capability to meet many of the assumed technical requirements of the tender, particularly concerning power generation (wind) and digital solutions.  However, gaps exist in energy storage and some aspects of environmental compliance, which would necessitate further clarification and detailed specification in a complete bid.  Commercial requirements are addressed separately.  A more thorough analysis would require precise tender specifications to determine full compliance and the relative importance of each clause.


In [34]:
# add the last response to the chat history
add_history_to_chat_single({'prompt': prompt_clause_by_clause, 'answer': responses['prompt_clause_by_close'].text}, "technical engineer", history_chat)

## Count the overall tokens

The total number of token can be computed by counting the tokens of history_chat, since the new responses have been appended to it for each call of Gemini.

In [35]:
print(f"{model.count_tokens(history_chat)=}")

model.count_tokens(history_chat)=total_tokens: 694248



# Alternative usage of Gemini: Context caching

As the entire dataset consists in JSON files, it could be cached using the Context caching functionality of Gemini:

https://ai.google.dev/gemini-api/docs/caching?lang=python

With context caching the system stores intermediate results from prior tender evaluations or company analyses, so if a similar query arises, the system can quickly retrieve relevant data and produce faster, more accurate responses.

In [53]:
# to ensure that no caching limit is exceeded, flush all the already available caches
from google.generativeai import caching

def delete_caches() -> None:
    for c in caching.CachedContent.list():
        print(f"Deleting cache named \"{c.display_name}\" ...")
        c.delete()
    print("All the caches have been deleted!")

delete_caches()

All the caches have been deleted!


In [54]:
import google.generativeai as genai
import time

@retry_on_failure(wait_time_seconds=10, max_retries=2)
def process_file_for_caching(file_path: str) -> genai.types.file_types.File | None:
    if os.path.exists(file_path):
        loaded_file = genai.upload_file(file_path)
        
        while loaded_file.state.name == "PROCESSING":
            print(f"Processing file {file_path} ...")
            time.sleep(2)
            loaded_file = genai.get_file(loaded_file.name)
        print(f"Processing file {file_path} completed. Available at {loaded_file.uri}")
        
        return loaded_file
    else:
        return None

# the process_file_for_caching() function will make the input file as available for caching
tenders_info_file = process_file_for_caching(tenders_info_json_path)
companies_info_file = process_file_for_caching(companies_info_json_path)

Processing file /kaggle/input/tenders-and-companies-websites/tenders_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/ehu2dijx3k4s
Processing file /kaggle/input/tenders-and-companies-websites/companies_info.json completed. Available at https://generativelanguage.googleapis.com/v1beta/files/575sc6y63jl2


In [55]:
cache_instructions = f"""
{system_prompt}
The information you need is in the JSON files you have access to.
"""

In [56]:
import datetime

@retry_on_failure(wait_time_seconds=60, max_retries=2)
def build_cache_from_contents(model_name: str ='gemini-1.5-flash-002', cache_name: str='cache', instructions: str="Use the information to answer", resources: list=[], minutes_available: int=10):
    cache = caching.CachedContent.create(
        model=model_name,
        display_name=cache_name, # used to identify the cache
        system_instruction=(instructions),
        contents=resources,
        ttl=datetime.timedelta(minutes=minutes_available),
    )
    
    model_with_cache = genai.GenerativeModel.from_cached_content(cached_content=cache)
    return model_with_cache

files_to_cache = [tenders_info_file, companies_info_file]
model_with_cache = build_cache_from_contents(cache_name='tenders and companies info', instructions=cache_instructions, resources=files_to_cache)

In [57]:
def list_available_caches():
    cache_list = list(caching.CachedContent.list())
    if len(cache_list) == 0:
        print("Empty cache!")
        return []
        
    for c in cache_list:
        print(f"Available cache with name \"{c.display_name}\"\n    model: {c.model}\n    created: {c.create_time}\n    expires: {c.expire_time}\n    tokens: {c.usage_metadata.total_token_count}")
        # print(c)
    return cache_list

caches = list_available_caches()

Available cache with name "tenders and companies info"
    model: models/gemini-1.5-flash-002
    created: 2024-11-27 21:57:26.629311+00:00
    expires: 2024-11-27 22:07:26.138870+00:00
    tokens: 668315


## Test if the cache works

In [58]:
def ask_gemini_with_cache_dummy_response(model, prompt: str=""):
    """
    Function to handle a possible error by Gemin when generating a response
    """
    try:
        response = model.generate_content(prompt)
    except Exception as exc:
        print(f"Execution in model with caching failed: {exc}")
        print("An empty response will be returned")
        
        class DummyResponse:
            def __init__(self):
                self.text = "MODEL WITH CACHING FAILED TO GENERATE RESPONSE"
        response = DummyResponse()
        
    return response

@retry_on_failure(wait_time_seconds=2, max_retries=5)
def ask_gemini_with_cache(model, prompt: str=""):
    """
    Wrapper for a Gemini model that uses cache
    """
    response = model.generate_content(prompt)
    return response

In [59]:
response_cached_files = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt="Describe the documents you have access to")

Markdown(response_cached_files.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Generate the clause-by-clause with cached content

In [60]:
response_cached_clause_by_clause = ask_gemini_with_cache_dummy_response(model=model_with_cache, prompt=f"Use the documents you have access to, to answer.\n {user_prompt}")

Markdown(response_cached_clause_by_clause.text)

Execution in model with caching failed: 500 Internal error encountered.
An empty response will be returned


MODEL WITH CACHING FAILED TO GENERATE RESPONSE

## Deleting the generated caches

In [61]:
delete_caches()

Deleting cache named "tenders and companies info" ...
All the caches have been deleted!
